In [1]:
import pyspark
import os
sc = pyspark.SparkContext(appName="BigDataProject")

In [2]:
my_dir = "./BooksDataset"

In [3]:
sc

<SparkContext master=local[*] appName=BigDataProject>

In [49]:
#Read Authors

authors = []

for file in os.listdir(my_dir):
    d = os.path.join(my_dir, file)
    if os.path.isdir(d):
        res = file
        authors.append(res)
print(authors)

['Arthur Conan Doyle', 'Charles Dickens', 'Daniel Defoe', 'Edith Wharton', 'Jane Austen', 'Joseph Conrad', 'Lewis Carroll', 'Louisa May Alcott', 'Voltaire', 'William Shakespeare']


In [50]:
#Read Books per Authors
books = [[] for _ in range(len(authors))]


for i in range(len(authors)):
    mydiraut = my_dir+"/"+authors[i]
    for file in os.listdir(mydiraut):
        d = os.path.join(mydiraut, file)
        books[i].append(file)

print(books)
    


[['Arthur Conan Doyle_Beyond the City.txt', 'Arthur Conan Doyle_The Adventure of the Cardboard Box.txt', 'Arthur Conan Doyle_The Adventure of the Dying Detective.txt', 'Arthur Conan Doyle_The Adventure of the Red Circle.txt', 'Arthur Conan Doyle_The Adventures of Gerard.txt', 'Arthur Conan Doyle_The Adventures of Sherlock Holmes.txt', 'Arthur Conan Doyle_The British Campaign in France and Flanders 1914.txt', "Arthur Conan Doyle_The Cabman's Story.txt", 'Arthur Conan Doyle_The Captain of the Polestar and Other Tales.txt', 'Arthur Conan Doyle_The case of Oscar Slater.txt'], ['Charles Dickens_A Christmas Carol.txt', 'Charles Dickens_American Notes for General Circulation.txt', 'Charles Dickens_Bardell v. Pickwick.txt', 'Charles Dickens_Bleak House.txt', "Charles Dickens_Charles Dickens' Children Stories.txt", 'Charles Dickens_David Copperfield.txt', 'Charles Dickens_Oliver Twist.txt', 'Charles Dickens_The Cricket on the Hearth.txt', 'Charles Dickens_The Life And Adventures Of Nicholas Nic

In [56]:
def wordCount(file_path):
    words = sc.textFile(file_path).filter(bool) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])) \
                .sortByKey(False)
    result = words

    return result


allWordsForAnAuthors = []

for i in range(len(authors)):
    for j in range(8,len(books[i])):
        input_filename = os.path.join(my_dir, authors[i])
        input_filename = os.path.join(input_filename, books[i][j])
        print(input_filename)
        wordsOfABook= wordCount(input_filename)
        
        allWordsForAnAuthors.append(wordsOfABook)


    allWordsForAnAuthors = sc.parallelize(allWordsForAnAuthors)
    allWordsForAnAuthors.flatMap(lambda x : (x[0], x[1])).reduceByKey(lambda v1, v2: v1+v2).sortByKey(False)
    print(f"{authors[i]} {allWordsForAnAuthors.take(100)}")
    allWordsForAnAuthors = []



./BooksDataset\Arthur Conan Doyle\Arthur Conan Doyle_The Captain of the Polestar and Other Tales.txt
./BooksDataset\Arthur Conan Doyle\Arthur Conan Doyle_The case of Oscar Slater.txt
Arthur Conan Doyle [[(5205, 'the'), (2620, 'and'), (2616, 'of'), (2096, 'a'), (2032, 'to'), (1837, 'i'), (1319, 'in'), (1271, 'was'), (1092, 'he'), (1048, 'his'), (1031, 'that'), (842, 'it'), (818, 'as'), (803, 'my'), (712, 'had'), (697, 'with'), (636, 'which'), (587, 'for'), (566, 'at'), (532, 'is'), (476, 'have'), (465, 'upon'), (423, 'from'), (405, 'you'), (394, 'be'), (393, 'but'), (370, 'by'), (367, 'on'), (335, 'this'), (330, ''), (325, 'me'), (322, 'her'), (317, 'were'), (307, 'there'), (304, 'one'), (281, 'an'), (277, 'not'), (269, 'all'), (266, 'him'), (257, 'when'), (257, 'been'), (254, 'we'), (252, 'some'), (252, 'she'), (245, 'so'), (244, 'out'), (235, 'into'), (225, 'would'), (222, 'who'), (221, 'up'), (217, 'their'), (214, 'they'), (205, 'or'), (202, 'no'), (198, 'could'), (182, 'are'), (176,

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 